In [2]:
import sqlite3

import pandas as pd

from interchange.persistence.database import Database

db = Database()

2025-11-26 14:44:31,121 :: PID 3424 :: TID 15016 :: database._create_connection :: Line 33 :: DEBUG :: Connected to SQLite database


In [25]:
path = "C:/Users/julio.cardenas/exchange_rate_202511_202511261442.csv"
df = pd.read_csv(path)

In [26]:
df = df[
    [
        "brand",
        "app_processing_date",
        "currency_from",
        "currency_to",
        "currency_from_code",
        "currency_to_code",
        "exchange_value",
    ]
]


In [27]:
# --- Normalización de texto ---
df["brand"] = df["brand"].str.upper()
df["currency_from"] = df["currency_from"].str.upper()
df["currency_to"] = df["currency_to"].str.upper()

# --- Forzar strings para los códigos ---
df["currency_from_code"] = df["currency_from_code"].astype(str)
df["currency_to_code"] = df["currency_to_code"].astype(str)

# Quitar decimales si aparecen como "971.0"
df["currency_from_code"] = df["currency_from_code"].str.replace(r"\.0$", "", regex=True)
df["currency_to_code"] = df["currency_to_code"].str.replace(r"\.0$", "", regex=True)

# Mantener los ceros a la izquierda: rellenar hasta 3 dígitos
df["currency_from_code"] = df["currency_from_code"].str.zfill(3)
df["currency_to_code"] = df["currency_to_code"].str.zfill(3)

# --- Renombrar fecha ---
df.rename(columns={"app_processing_date": "rate_date"}, inplace=True)

# --- Convertir rate_date a fecha YYYY-MM-DD ---
df["rate_date"] = pd.to_datetime(df["rate_date"]).dt.strftime("%Y-%m-%d")


In [28]:
df

,brand,rate_date,currency_from,currency_to,currency_from_code,currency_to_code,exchange_value
0,MASTERCARD,2025-11-01,AFN,ALL,971,008,1.266326
1,MASTERCARD,2025-11-01,AFN,DZD,971,012,1.951115
2,MASTERCARD,2025-11-01,AFN,AOA,971,973,13.785975
3,MASTERCARD,2025-11-01,AFN,ARS,971,032,21.760395
4,MASTERCARD,2025-11-01,AFN,AMD,971,051,5.704849
...,...,...,...,...,...,...,...
1211533,MASTERCARD,2025-11-25,ZWG,VUV,924,548,4.724482
1211534,MASTERCARD,2025-11-25,ZWG,VES,924,928,9.482059
1211535,MASTERCARD,2025-11-25,ZWG,VND,924,704,1029.798354
1211536,MASTERCARD,2025-11-25,ZWG,YER,924,886,9.750770


In [30]:
conn = sqlite3.connect("persistence/backup/interchange.db")
cursor = conn.cursor()

insert_sql = """
INSERT INTO exchange_rate (
    brand,
    rate_date,
    currency_from,
    currency_to,
    currency_from_code,
    currency_to_code,
    exchange_value
)
VALUES (?, ?, ?, ?, ?, ?, ?)
"""

cursor.executemany(
    insert_sql,
    df[
        [
            "brand",
            "rate_date",
            "currency_from",
            "currency_to",
            "currency_from_code",
            "currency_to_code",
            "exchange_value",
        ]
    ].values.tolist(),
)

conn.commit()
conn.close()